# Extract price and demand data from AEMO
- Using the aggregated price and demand website: https://aemo.com.au/energy-systems/electricity/national-electricity-market-nem/data-nem/aggregated-data
- Download the CSV files to a specified location
- Add data to a local SQLite database
- Delete all CSV files
- Extract data for use

## Import modules

In [3]:
import time
import requests
import os
import pandas as pd
import sqlite3
from datetime import datetime
import pytz
import hvplot.pandas

## Specify data to be downloaded and location to store

In [5]:
# Specify which states you want the price and demand data for 
states = ['QLD1','NSW1','VIC1','SA1','TAS1']  # Change this to the desired state, options are 'QLD1','NSW1','VIC1','SA1','TAS1', multiple can be selected

# Specify the start and end year/month you want data for
start_month = "2015-02-01" # must be in format "YYYY-MM-DD"
end_month = "2025-05-01" # go to the next month after what you want


states_dic = {} # Create an empty dictionary to store the the download data
date_range = pd.date_range(start=start_month, end=end_month, freq='ME')# Monthly
yyyymm_list = [date.strftime('%Y%m') for date in date_range]

# This is the base url used for the csv download 
base_url = 'https://aemo.com.au/aemo/data/nem/priceanddemand/'

## Download all CSV's

In [8]:
for year_month in yyyymm_list:
    for state in states:
        download_url = f'{base_url}PRICE_AND_DEMAND_{year_month}_{state}.csv'
        # Specify the download location
        download_folder = r'C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA'  # Change this to your desired folder
        file_name = f'PRICE_AND_DEMAND_{year_month}_{state}.csv'
        file_path = os.path.join(download_folder, file_name)
        
        # Headers to mimic a browser request
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.64',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive'
        }
        
        # Download the file
        response = requests.get(download_url, headers=headers)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f'File downloaded successfully and saved to {file_path}')
        else:
            print(f'Failed to download file. Status code: {response.status_code}')
    
        states_dic[f"{state}{year_month}"]=file_path

File downloaded successfully and saved to C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_QLD1.csv
File downloaded successfully and saved to C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_NSW1.csv
File downloaded successfully and saved to C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_VIC1.csv
File downloaded successfully and saved to C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_SA1.csv
File downloaded successfully and saved to C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_TAS1.csv
File downloaded successfully and saved to C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201503_QLD1.csv
File downloaded successfully and saved to C:\Users\Lachlan Ryan\OneDrive - VP

## Combining csv's and preparing to store into database

In [10]:
combined_df = pd.DataFrame()
for key, value in states_dic.items():
    price_demand_csv = pd.read_csv(value)
    combined_df = pd.concat([combined_df, price_demand_csv], ignore_index=True)

combined_df.SETTLEMENTDATE = pd.to_datetime(combined_df.SETTLEMENTDATE)
today = pd.Timestamp(datetime.now(pytz.utc).astimezone(pytz.timezone('Australia/Brisbane'))).normalize()
df_to_database = combined_df[combined_df['SETTLEMENTDATE'].dt.normalize() != today]

df_to_database 

,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
0,QLD1,2015-02-01 00:30:00,5600.32,18.07,TRADE
1,QLD1,2015-02-01 01:00:00,5429.88,16.45,TRADE
2,QLD1,2015-02-01 01:30:00,5268.36,15.75,TRADE
3,QLD1,2015-02-01 02:00:00,5186.41,15.11,TRADE
4,QLD1,2015-02-01 02:30:00,5110.83,13.73,TRADE
...,...,...,...,...,...
2456155,TAS1,2025-04-22 23:40:00,1044.86,80.13,TRADE
2456156,TAS1,2025-04-22 23:45:00,1041.21,80.13,TRADE
2456157,TAS1,2025-04-22 23:50:00,1039.19,80.13,TRADE
2456158,TAS1,2025-04-22 23:55:00,1037.52,80.13,TRADE


## Pass data into the database

In [12]:
# Use 'with' to connect to the SQLite database and automatically close the connection when done
with sqlite3.connect(r'C:\\Users\\Lachlan Ryan\\OneDrive - VPP Partners\\Documents\\Python Scripts\\NEMDATA\\PRICE_AND_DEMAND_NEM.db') as connection:

    # Create a cursor object
    cursor = connection.cursor()

    # Create a new table
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS price_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        REGION TEXT,
        SETTLEMENTDATE TEXT,
        TOTALDEMAND REAL,
        RRP REAL,
        PERIODTYPE TEXT
    );
    '''
    
    # Execute the SQL command
    cursor.execute(create_table_query)

    # Commit the changes
    connection.commit()

    df_to_database['SETTLEMENTDATE'] = df_to_database['SETTLEMENTDATE'].astype(str)
    df_to_database.to_sql('price_data', connection, if_exists='append', index=False)


    delete_duplicates_quey = '''
    WITH CTE AS (
        SELECT 
            id,
            REGION,
            SETTLEMENTDATE,
            TOTALDEMAND,
            RRP,
            PERIODTYPE,
            ROW_NUMBER() OVER (
                PARTITION BY REGION, SETTLEMENTDATE, TOTALDEMAND, RRP, PERIODTYPE 
                ORDER BY id
            ) AS row_num
        FROM price_data
    )
    DELETE FROM price_data
    WHERE id IN (
        SELECT id
        FROM CTE
        WHERE row_num > 1
    );
    '''
    # Execute the SQL command
    cursor.execute(delete_duplicates_quey)

    # Commit the changes
    connection.commit()

    # Print a confirmation message
    print("Table 'price_data' created successfully!")

Table 'price_data' created successfully!


## Deleting all csv's

In [14]:
# Loop through the dictionary and delete each file
for key, file_path in states_dic.items():
    try:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

print("File deletion process completed.")

Deleted: C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_QLD1.csv
Deleted: C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_NSW1.csv
Deleted: C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_VIC1.csv
Deleted: C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_SA1.csv
Deleted: C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201502_TAS1.csv
Deleted: C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201503_QLD1.csv
Deleted: C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201503_NSW1.csv
Deleted: C:\Users\Lachlan Ryan\OneDrive - VPP Partners\Documents\Python Scripts\NEMDATA\PRICE_AND_DEMAND_201503_VIC1.csv
Deleted: C:\Users\Lachlan Ryan\On

## Reading from the database

In [87]:
# Update query as required, some examples below:
query = '''
SELECT * FROM price_data
WHERE REGION = 'SA1' AND SETTLEMENTDATE > '2024-01-01' AND SETTLEMENTDATE < '2025-01-01'
'''

#Examples
"""

SELECT * FROM price_data


SELECT * FROM price_data
WHERE REGION = 'TAS1'

SELECT * FROM price_data
WHERE REGION = 'SA1' AND SETTLEMENTDATE > '2024-01-01' AND SETTLEMENTDATE < '2025-01-01'

"""

with sqlite3.connect(r'C:\\Users\\Lachlan Ryan\\OneDrive - VPP Partners\\Documents\\Python Scripts\\NEMDATA\\PRICE_AND_DEMAND_NEM.db') as connection:

    df_from_database = pd.read_sql_query(query, connection)

df_from_database.SETTLEMENTDATE = pd.to_datetime(df_from_database.SETTLEMENTDATE)
df_from_database.set_index('SETTLEMENTDATE',inplace=True)
df_from_database.drop(columns='id',inplace=True)

In [88]:
df_from_database

,REGION,TOTALDEMAND,RRP,PERIODTYPE
SETTLEMENTDATE,,,,
2024-01-01 00:00:00,SA1,1296.67,54.02,TRADE
2024-01-01 00:05:00,SA1,1293.98,54.42,TRADE
2024-01-01 00:10:00,SA1,1300.29,68.20,TRADE
2024-01-01 00:15:00,SA1,1302.81,56.17,TRADE
2024-01-01 00:20:00,SA1,1311.46,52.29,TRADE
...,...,...,...,...
2024-12-31 23:35:00,SA1,1354.54,129.99,TRADE
2024-12-31 23:40:00,SA1,1375.14,130.16,TRADE
2024-12-31 23:45:00,SA1,1369.37,141.80,TRADE


In [91]:
df_from_database[['RRP']].hvplot(width=1500,height=600)

:Curve   [SETTLEMENTDATE]   (RRP)